Для посева этих культур используются три участка посевных площадей размером S1, S2, S3(га). Характеристики себестоимости (руб/т) выращивания тонны і-й культуры на з-м участке, а
также площади участков представлены в табл. 2. 

В случае нехватки посевных площадей для выполнения плана по кормовым культурам (см.табл. 1), имеется возможность закупки необходимого количества культур по цене, приведенной в табл. 3. Требуется найти план посева (распределения культур по участкам), оптимальный с точки зрения минимизации себестоимости, и провести исследование решения:
 - выяснить размер свободных площадей на участках при точном выполнении планов;
 - выяснить возможности дополнительного посева культур с полным использованием площадей участков; ﻿﻿предположив, что существует возможность затопления участка 2 (ведущая к полной потере урожая с него), проанализировать себестоимость «оптимистичной» и «пессимистич-ной» стратегий распределения культур по участкам. Под «оптимистичной» стратегией здесь понимается такая, в соответствии с которой при посеве игнорируется возможность затопления, а под «пессимистичной» - такая, при которой посев на рискованный участок вообще не производится;
 - формализовать задачу минимизации ожидаемой себестоимости при известной вероятности затопления, построить график изменения ожидаемой себестоимости при вероятности затопления от 0 до 0.5.

Решим задачу методом линейного программироввания

In [267]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.optimize import linprog


plan = np.array([666, 527, 514, 393])
cultures = np.array([
    [19.0, 16.0, 0.018, 0.25],
    [15.0, 8.4, 0.017, 0.3],
    [19.0, 17.0, 0.025, 0.25],
    [21.0, 14.9, 0.056, 0.3]
])

# Себестоимость выращивания культур на участках
costs = np.array([
    [28, 37, 61],
    [45, 50, 32],
    [46, 47, 44],
    [54, 24, 36]
])
#Цена кормовой культуры
prices = np.array([64, 53, 50, 57])

areas = np.array([100, 150, 200])


#функция минимизации себестоимости посева культур на участках
def objective(x):
    x = x.reshape(3, 4)
    total_cost = np.sum(costs.T * x, axis=1) * areas
    return np.sum(total_cost)


x0 = np.ones(12)


Ограничения:

In [268]:
bounds = [(0, None)] * 12
areas_constraint = {'type': 'eq', 'fun': lambda x: np.sum(x.reshape(3, 4), axis=1) - areas}
constraints = [areas_constraint]

Оптимальный план посева и минимальная себестоимость:

In [269]:
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)
optimal_plan = result.x.reshape(costs.shape)
min_cost = result.fun

print("Оптимальный план:\n", optimal_plan)
print("Минимальная себестоимость:", min_cost)

Оптимальный план:
 [[9.99999943e+01 6.56355903e-07 1.03153239e-06]
 [4.03550166e-06 1.45976327e-12 5.90969696e-06]
 [4.22132042e-06 1.49999991e+02 1.33223235e-05]
 [1.99999992e+02 5.62837501e-07 5.64442172e-12]]
Минимальная себестоимость: 2100000.1695842654


Размер свободных площадей на участках:

In [270]:
free_areas = areas - np.sum(optimal_plan, axis=0)
free_areas[free_areas < 0] = 0 # заменяем отрицательные значения на ноль
print("Размер свободных площадей на участках: ", free_areas)

Размер свободных площадей на участках:  [0.00000000e+00 7.50462468e-06 1.99999980e+02]


Возможности дополнительного посева культур:

In [271]:
additional_planting = (free_areas.reshape(-1, 1) / costs.mean(axis=1).reshape(1, -1)).T
print("Возможности дополнительного посева культур: ")
print(additional_planting)

Возможности дополнительного посева культур: 
[[0.00000000e+00 1.78681540e-07 4.76190428e+00]
 [0.00000000e+00 1.77274599e-07 4.72440897e+00]
 [0.00000000e+00 1.64334847e-07 4.37956160e+00]
 [0.00000000e+00 1.97490123e-07 5.26315736e+00]]


Оптимистичная стратегия

In [272]:
optimistic_plan = plan.copy()
optimistic_plan[1] += optimistic_plan[2]
optimistic_plan[2] = 0
optimistic_cost = np.sum(costs * optimistic_plan[:, np.newaxis], axis=1).astype('float64') * plan
optimistic_cost += np.sum(additional_planting.reshape(-1, 1) * prices, axis=0)
print("Себестоимость при оптимистичной стратегии:", np.sum(optimistic_cost))


Себестоимость при оптимистичной стратегии: 143172615.9033761


Пессимичтичная стратегия

Для того, чтобы написать пессимистичную стратегию распределения культур по участкам, необходимо учесть возможность затопления участка 2, ведущей к полной потере урожая с него. Следовательно, пессимистичная стратегия будет заключаться в том, что на участок 2 не будет производиться посев.

In [273]:
pessimistic_plan = plan.copy()
pessimistic_plan[2] = 0
pessimistic_cost = np.sum(costs * pessimistic_plan[:, np.newaxis], axis=1).astype('float64') * plan
pessimistic_cost += np.sum(additional_planting.reshape(-1, 1) * prices, axis=0)

print("Себестоимость при пессимистичной стратегии:", np.sum(pessimistic_cost))


Себестоимость при пессимистичной стратегии: 108771109.90337609
